In [ ]:
!pip install -U transformers
!pip install accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 47.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 32.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 110.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 83.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 3.6 MB/s eta 0:00:00


In [ ]:
import numpy as np
import pandas as pd
import operator
import os
import string
import re
import random
import sys
import platform

import torch
import torch.optim as optim
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

from PIL import Image

from transformers import Blip2Processor, Blip2ForConditionalGeneration

from tqdm.auto import tqdm

In [ ]:
test_df = pd.read_csv('/content/drive/MyDrive/Dacon_multimodal_for_zeroshot/test.csv')
test_img_path = '/content/drive/MyDrive/Dacon_multimodal_for_zeroshot/image/'
sample_submission = pd.read_csv('/content/drive/MyDrive/Dacon_multimodal_for_zeroshot/sample_submission.csv')

In [ ]:
number_words_dict = {
    "zero": 0,
    "one": 1,
    "two": 2,
    "three": 3,
    "four": 4,
    "five": 5,
    "six": 6,
    "seven": 7,
    "eight": 8,
    "nine": 9,
}

def convert_word_to_number(text):
    words = text.lower().split()
    for word in words:
        if word in number_words_dict:
            return str(number_words_dict[word])
    return text

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"

processor = Blip2Processor.from_pretrained("Salesforce/blip2-flan-t5-xl")
model = Blip2ForConditionalGeneration.from_pretrained("Salesforce/blip2-flan-t5-xl")
model.to(device)

final_submission = sample_submission.copy()

for idx, row in tqdm(test_df.iterrows()):
    img_name = os.path.join(test_img_path, row['image_id'] + '.jpg')  # 이미지
    image = Image.open(img_name).convert('RGB')

    prompt = "Your task is to answer a question that is knowledge-based or may require compositional reasoning. If the answer is represented as a numerical word, just output it as an Arabic numeral. Question : {} Short answer:".format(row['question'])
    inputs = processor(images=image, text=prompt, return_tensors="pt").to(device)

    generated_ids = model.generate(**inputs)
    generated_text = processor.batch_decode(generated_ids, skip_special_tokens=True)[0].strip()

    if "yes" in generated_text:
        generated_text = "yes"
    elif "no" in generated_text:
        generated_text = "no"

    generated_text = convert_word_to_number(generated_text)

    final_submission.loc[final_submission['ID'] == row['ID'], 'answer'] = generated_text

    if idx%100 == 0:
        final_submission.to_csv('/content/drive/MyDrive/Dacon_multimodal_for_zeroshot/submission_zeroshot_blip2.csv', index=False)

final_submission.to_csv('/content/drive/MyDrive/Dacon_multimodal_for_zeroshot/submission_zeroshot_blip2.csv', index=False)

In [ ]:
final_submission

,ID,answer
0,TEST_00000,no
1,TEST_00001,on the beach
2,TEST_00002,red
3,TEST_00003,white
4,TEST_00004,no
...,...,...
40474,TEST_40474,a digital camera
40475,TEST_40475,graffiti
40476,TEST_40476,air
40477,TEST_40477,red
